In [19]:
import cv2

from ultralytics import YOLO


In [20]:
classe_choisie = "2x4_Jaune"

In [21]:
# chargement model onnx
# model= YOLO("/home/dim/clone_repo/BrickSearch/outputs/output_models/y12m_100_640_c_10_10_05/weights/best.onnx")

#chargement model onnx
model = YOLO("/home/dim/clone_repo/BrickSearch/outputs/output_models/y12m_100_640_c_10_10_05/weights/best.engine")


WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [22]:
source = '/home/dim/clone_repo/BrickSearch/data/raw/videos/lego_video_test.mp4'

path_video_output = '/home/dim/clone_repo/BrickSearch/outputs/video_annotees'

In [ ]:

# Open the video file
video_path = source
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run  tracking on the frame, persisting tracks between frames
        results = model(
                                    source=frame,
                                    conf=0.8, 
                                    iou=0.5, 
                                    classes=23, 
                                )

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLO12 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

Loading /home/dim/clone_repo/BrickSearch/outputs/output_models/y12m_100_640_c_10_10_05/weights/best.engine for TensorRT inference...


[03/27/2025-17:35:59] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[03/27/2025-17:35:59] [TRT] [I] Loaded engine size: 95 MiB
[03/27/2025-17:35:59] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +1, GPU +93, now: CPU 3, GPU 531 (MiB)

0: 640x640 1 2x4_Jaune, 15.9ms
Speed: 5.0ms preprocess, 15.9ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 2x4_Jaune, 15.6ms
Speed: 3.0ms preprocess, 15.6ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 2x4_Jaune, 16.3ms
Speed: 2.7ms preprocess, 16.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 2x4_Jaunes, 16.2ms
Speed: 2.7ms preprocess, 16.2ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 2x4_Jaunes